## Proceso de Entrenamiento de Modelo y Predicción de Valores
1. **Entrenamiento**: ...
2. **Predicción**: ...

In [6]:
######## CONFIGURACIONES PRELIMINARES ##########
import os
import pandas as pd
import warnings
import io
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.5f}'.format #evita que muestre notacion cientifica
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
module_path = os.path.abspath(os.path.join('../../'))
import sys
if module_path not in sys.path:
    sys.path.append(module_path)
from src.modeling import train_predict

In [ ]:
%%time
X_train, X_test, y_train, y_test, cols_for_model, lgbm_model = train_predict.train_lgbm_model()

In [ ]:
%%time
import pandas as pd
series_features_mes = pd.read_parquet('../../data/historico/df_series_features_mes.parquet')
reporte_predicciones = predicciones_mes(series_features_mes)